# Tensorflow Model Optimization Toolkit
Is a suite of techniques that allows to optimize machine learning models for deployment and execution. The notebook shows the difference between simple tensorflow models and the same models after applying the __Model Optimization Toolkit(MOT)__. More about this toolkit can find [here](https://www.tensorflow.org/performance/model_optimization) and [here](https://www.tensorflow.org/performance/post_training_quantization). Two types of models will be used:
* __VGG16__
* __ResNet34__

These models have totally dissimilar architecture, the question is how Model Optimization Toolkit cope with them. These models will be train on two datasets:
* __CIFAR10__
* __CIFAR100__

In [6]:
import sys
sys.path.append('../../')

import tensorflow as tf
from copy import deepcopy

from dataset.opensets import CIFAR10, CIFAR100
from dataset.models.tf import VGG16, ResNet34
from dataset import Pipeline, B, V, C

For training and testing model you need to create pipelines, to avoid a large amount of repetitive code, combine the creation of the pipeline into a separate function named __create_pipelines.__

In [86]:
def create_pipelines(model, data, name, config):
    """Created pipelines with given parameters.
    Parameters
    ----------
    model : Dataset model
        model for training and testing
    data : Dataset
        the data on which the model will be train
    name : str
        the name of pipelines
    config : dict
        configuration dict to current model
    
    Returns
    -------
    train_pipeline : Dataset pipeline
        pipeline for training current model
    test_pipeline : Dataset pipeline
        pipeline for testing current model
    """
    train_pipeline = ((
        Pipeline()
        .init_variable('loss', init_on_each_run=list)
        .init_variable('loss_history', init_on_each_run=list)  
        .init_model('dynamic', model, name, config=config)
        .to_array()
        .train_model(name, fetches='loss', 
                     feed_dict={'images': B('images'), 'labels': B('labels')},
                     save_to=V('loss'))
        .update_variable('loss_history', V('loss'), mode='a')
    ) << data.train)


    test_pipeline = ((
        Pipeline()
        .import_model(name, train_pipeline)
        .init_variable('predictions', init_on_each_run=list)
        .init_variable('metrics', init_on_each_run=None)
        .to_array()
        .predict_model(name, fetches='predictions', 
                       feed_dict={'images': B('images'), 'labels': B('labels')},
                       save_to=V('predictions'))
        .gather_metrics('class', targets=B('labels'), predictions=V('predictions'),
                        fmt='logits', axis=-1, save_to=V('metrics'), mode='w')
    ) << data.test)
    return train_pipeline, test_pipeline

## CIFAR 10
### Define a dataset
Firstly create a dataset instance:

In [69]:
cifar10 = CIFAR10()

Extracting...
Extracted
Downloaded cifar-100-python.tar.gz
Extracting...
Extracted


### Models creation

Configurate the parameters for certain models.

In [88]:
cifar10_config = {
    'inputs': dict(images=dict(shape=B('image_shape')),
                   labels=dict(classes=10, transform='ohe', name='targets')),
    'input_block/inputs': 'images',
    'loss': 'ce',
    'optimizer': 'Adam',
}

By function described above, create pipelines for each models.

In [89]:
vgg_train_10_ppl, vgg_test_10_ppl = create_pipelines(VGG16, cifar10, 'cifar10_vgg16', cifar10_config)
resnet_train_10_ppl, resnet_test_10_ppl = create_pipelines(ResNet34, cifar10,'cifar10_resnet34', cifar10_config)

### Training models

Now you can train models by running the pipelines.

In [ ]:
vgg_train_ppl.run(50, shuffle=True, n_epochs=3, drop_last=True, bar=True)
resnet_train_ppl.run(50, shuffle=True, n_epochs=3, drop_last=True, bar=True)

### Models Optimization Toolkit
Now let's apply the MOT to each trained on cifar10 models. 

In [ ]:
converter=tf.contrib.lite.TocoConverter.from_saved_model(saved_model_dir)
converter.post_training_quantize=True
tflite_quantized_model = converter.convert()
# open("quantized_model.tflite", "wb").write(tflite_quantized_model)

### Analysis of results

## CIFAR 100
### Define a dataset
Firstly create a dataset instance, in this turn is cifar100:

In [ ]:
cifar100 = CIFAR100()

### Models creation
As same as with cifar10, you have to create a configuration dictionary with current parameters.

In [92]:
cifar100_config = {
    'inputs': dict(images=dict(shape=B('image_shape')),
                   labels=dict(classes=100, transform='ohe', name='targets')),
    'input_block/inputs': 'images',
    'loss': 'ce',
    'optimizer': 'Adam',
}

The function __create_pipelines__ will be useful again, it helps you to create a new instance of pipelines with the cifar100 inside.

In [91]:
vgg_train_100_ppl, vgg_test_100_ppl = create_pipelines(VGG16, cifar100, 'cifar100_vgg16', cifar100_config)
resnet_train_100_ppl, resnet_test_100_ppl = create_pipelines(ResNet34, cifar100,'cifar100_resnet34', cifar100_config)

### Training models
After all preparations you are able to train your models, let's do it.

In [ ]:
vgg_train_ppl.run(50, shuffle=True, n_epochs=3, drop_last=True, bar=True)
resnet_train_ppl.run(50, shuffle=True, n_epochs=3, drop_last=True, bar=True)

### Models Optimization Toolkit
Now let's apply the MOT to each trained on cifar100 models. 

In [ ]:
converter=tf.contrib.lite.TocoConverter.from_saved_model(saved_model_dir)
converter.post_training_quantize=True
tflite_quantized_model = converter.convert()
# open("quantized_model.tflite", "wb").write(tflite_quantized_model)

### Analysis of results